In [25]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [26]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [27]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_citibike_data

from_year = 2024
# to_year = datetime.now().year
to_year = 2025
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
for year in range(from_year, to_year+1):

    rides_one_year = load_and_process_citibike_data(year)

    chunks.append(rides_one_year)
    break

# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2024 to 2025
📁 File already exists: citi_bike_rides_raw_2024_01.parquet
📖 Loading data from: citi_bike_rides_raw_2024_01.parquet
🔍 Filtering data for 2024-01...
✅ Saved filtered data to: /Users/yashmathur/Documents/MS_DS/Python_Spring_25/CDA_500/Test_Final/data/processed/citi_bike_rides_processed_2024_01.parquet
📁 File already exists: citi_bike_rides_raw_2024_02.parquet
📖 Loading data from: citi_bike_rides_raw_2024_02.parquet
🔍 Filtering data for 2024-02...
✅ Saved filtered data to: /Users/yashmathur/Documents/MS_DS/Python_Spring_25/CDA_500/Test_Final/data/processed/citi_bike_rides_processed_2024_02.parquet
📁 File already exists: citi_bike_rides_raw_2024_03.parquet
📖 Loading data from: citi_bike_rides_raw_2024_03.parquet
🔍 Filtering data for 2024-03...
✅ Saved filtered data to: /Users/yashmathur/Documents/MS_DS/Python_Spring_25/CDA_500/Test_Final/data/processed/citi_bike_rides_processed_2024_03.parquet
📁 File already exists: citi_bike_rides_raw_2024_04.parquet
📖 

In [28]:
rides

,started_at,start_station_id
0,2024-01-22 18:43:19.012,7954.12
1,2024-01-11 19:19:18.721,6771.13
2,2024-01-30 19:17:41.693,5659.11
3,2024-01-27 11:27:01.759,6771.13
4,2024-01-16 15:15:41.000,7443.01
...,...,...
44233324,2024-12-21 19:08:07.762,4386.07
44233325,2024-12-17 18:59:56.045,6925.09
44233326,2024-12-18 06:44:59.074,6717.06
44233327,2024-12-20 20:53:58.339,4019.06


In [29]:
rides.shape

(44233329, 2)

In [30]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [31]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26352 entries, 0 to 26351
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   hour              26352 non-null  datetime64[ns]
 1   start_station_id  26352 non-null  object        
 2   ride_count        26352 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 617.8+ KB


In [32]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')  
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)  
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-05-11 08:49:50,456 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:49:50,492 INFO: Initializing external client
2025-05-11 08:49:50,493 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 08:49:51,132 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Successfully connected to Hopsworks project: Citi_Bike_3010


In [33]:
feature_store = project.get_feature_store()

In [34]:
FEATURE_GROUP_NAME = "citi_bike_time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [35]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time series data at hourly freaquency",
    primary_key=["start_station_id", "hour"],
    event_time="hour"
)

In [36]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 26352/26352 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: citi_bike_time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1231009/jobs/named/citi_bike_time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('citi_bike_time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)